In [245]:
%matplotlib inline
import numpy as np
import os #helpful for manuplating files,directories
import math #helpful for any mathematical computations
import shutil #helpful for moving files from one directory to another
import matplotlib
import glob #works with image paths
from matplotlib import pyplot as plt
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.models import load_model
from keras.layers import GlobalAveragePooling2D, Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation
from keras.preprocessing.image import ImageDataGenerator

In [109]:
#count number of images 0-has atopic dematitis 1-doesn't have atopic dematitis
ROOT_DIRECTORY = "E:/datasets/brain_tumor_dataset"
num_of_images = {}

for dir in os.listdir(ROOT_DIRECTORY):#directories with the root directory
    num_of_images[dir] = len(os.listdir(os.path.join(ROOT_DIRECTORY, dir)))
    


In [110]:
num_of_images.items()

dict_items([('no', 17), ('yes', 17)])

In [60]:
#split the data such that
#70% for Train Data
#15% for validation
#15% for testing

#create train folder


In [98]:
def CreateFolder(path, split):
    if not os.path.exists("E:/datasets"+path):
        os.mkdir("E:/datasets/"+path)
        
        for dir in os.listdir(ROOT_DIRECTORY):
            os.makedirs("E:/datasets/"+path+"/"+dir)
            for img in np.random.choice(a = os.listdir(os.path.join(ROOT_DIRECTORY, dir)),
                                    size = (math.floor(split*num_of_images[dir])-5),
                                   replace=False ):
                O = os.path.join(ROOT_DIRECTORY,dir,img)
                D = os.path.join("E:/datasets/",path,dir)
                shutil.copy(O,D)
                os.remove(O)
    else:
        print(f"{path} Folder exists")

In [106]:
#CreateFolder("train1", 0.7)

In [107]:
#CreateFolder("Val1", 0.15)

In [108]:
#CreateFolder("test1", 0.15)

Model

In [307]:
#keras underlying framework for tensorflow
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten,Dense, BatchNormalization, GlobalAvgPool2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import keras
import tensorflow as tf
from tensorflow.keras import models, layers


In [309]:
#CNN model
#kernel_size-size of convulational filter
#activation_function-add up some non linearity
#conv layer is basically the features of the image
#add padding = 'same' to leave image size constant

model = models.Sequential([
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape = (224,224,3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(rate=0.25),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(rate=0.25),
    layers.Dense(1, activation='sigmoid'),
])

model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_38 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 111, 111, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_39 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 54, 54, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_40 (Conv2D)          (None, 52, 52, 64)        36928     
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 26, 26, 64)     

In [328]:
#binary crossentropy since its a binary problem i.e, has or doesn't have
loss_function = 'binary_crossentropy'
model.compile(optimizer ='adam',loss=loss_function, metrics=['accuracy'])

Data preparation and augmentation

In [329]:
def preprocessingImage1(path):
    
    image_data = ImageDataGenerator(zoom_range= 0.2,
                                   shear_range= 0.2,
                                   rescale= 1/255,
                                   horizontal_flip= True)
    image = image_data.flow_from_directory(directory = path,
                                          target_size = (224,224),
                                          batch_size = 32,
                                          class_mode = 'binary')
    return image

In [330]:
path = "E:/datasets/train1"
train_data = preprocessingImage1(path)

Found 2256 images belonging to 2 classes.


In [331]:
def preprocessingImage2(path):
    
    image_data = ImageDataGenerator(rescale= 1/255)
    image = image_data.flow_from_directory(directory = path,
                                          target_size = (224,224),
                                          batch_size = 32,
                                          class_mode = 'binary')
    return image

In [332]:
path = "E:/datasets/test1"
test_data = preprocessingImage2(path)

Found 474 images belonging to 2 classes.


In [333]:
path = "E:/datasets/Val1"
val_data = preprocessingImage2(path)

Found 474 images belonging to 2 classes.


In [334]:
""""
early stopping-incase results come early,
avoids wasting resources due to 
further training of all other perimeters
"""

from keras.callbacks import ModelCheckpoint, EarlyStopping

#hyperparameters
#early stopping
jamnyo = EarlyStopping(monitor="val_accuracy", min_delta=0.01, patience= 3, verbose = 1, mode = 'auto')

#model check point
george = ModelCheckpoint(monitor="val_accuracy", filepath="E:/datasets/atopicdema.h5",verbose= 1, save_best_only=True , mode = 'auto')

x = [jamnyo,george]

In [335]:
history = model.fit(
    train_data,
    batch_size=32,
    validation_data=val_data,
    verbose=1,
    epochs=20,
)

Epoch 1/20


AttributeError: in user code:

    File "C:\Users\XerxesCodes\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\XerxesCodes\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\XerxesCodes\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\XerxesCodes\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 863, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "C:\Users\XerxesCodes\AppData\Roaming\Python\Python39\site-packages\keras\optimizer_v2\optimizer_v2.py", line 532, in minimize
        return self.apply_gradients(grads_and_vars, name=name)
    Fi

Model Training